# Pipeline de Treinamento do Modelo de Otimização de Entrevistas

Este notebook implementa o pipeline completo de Machine Learning para o sistema de otimização de entrevistas com modelo aprimorado:

1. **Carregamento e Junção de Dados** - Importar vagas, candidatos e prospects da base completa
2. **Engenharia de Features Avançadas** - Criar features inteligentes que aproveitam todos os dados disponíveis
3. **Treinamento do Modelo Aprimorado** - Treinar um classificador com 7 features avançadas
4. **Avaliação e Teste** - Validar a performance e testar com exemplos reais
5. **Serialização** - Salvar o modelo treinado para produção

In [1]:
# Imports das bibliotecas necessárias
import pandas as pd
import numpy as np
import json
import os
from pathlib import Path

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Carregamento e Exploração dos Dados

Vamos carregar os três arquivos JSON e entender a estrutura dos dados.

In [2]:
# Definir caminhos dos arquivos
data_path = Path("../data")

# Usar a base completa de dados para treinamento
vagas_dev_path = data_path / "vagas.json"
applicants_dev_path = data_path / "applicants.json"
prospects_dev_path = data_path / "prospects.json"

# Função para carregar dados JSON
def load_json_data(file_path):
    """Carrega dados de um arquivo JSON"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e:
        print(f"Erro ao carregar {file_path}: {e}")
        return {}

# Carregar base completa de dados
print("Carregando base completa de dados...")
vagas_data = load_json_data(vagas_dev_path)
applicants_data = load_json_data(applicants_dev_path)
prospects_data = load_json_data(prospects_dev_path)

print(f"Vagas carregadas: {len(vagas_data)}")
print(f"Candidatos carregados: {len(applicants_data)}")
print(f"Prospects carregados: {len(prospects_data)}")

# Explorar estrutura dos dados
print("\n=== Estrutura das Vagas ===")
if vagas_data:
    vaga_exemplo = list(vagas_data.values())[0]
    print("Chaves principais:", list(vaga_exemplo.keys()))
    print("Informações básicas:", list(vaga_exemplo.get('informacoes_basicas', {}).keys()))
    print("Perfil vaga:", list(vaga_exemplo.get('perfil_vaga', {}).keys()))

Carregando base completa de dados...
Vagas carregadas: 14081
Candidatos carregados: 42482
Prospects carregados: 14222

=== Estrutura das Vagas ===
Chaves principais: ['informacoes_basicas', 'perfil_vaga', 'beneficios']
Informações básicas: ['data_requicisao', 'limite_esperado_para_contratacao', 'titulo_vaga', 'vaga_sap', 'cliente', 'solicitante_cliente', 'empresa_divisao', 'requisitante', 'analista_responsavel', 'tipo_contratacao', 'prazo_contratacao', 'objetivo_vaga', 'prioridade_vaga', 'origem_vaga', 'superior_imediato', 'nome', 'telefone']
Perfil vaga: ['pais', 'estado', 'cidade', 'bairro', 'regiao', 'local_trabalho', 'vaga_especifica_para_pcd', 'faixa_etaria', 'horario_trabalho', 'nivel profissional', 'nivel_academico', 'nivel_ingles', 'nivel_espanhol', 'outro_idioma', 'areas_atuacao', 'principais_atividades', 'competencia_tecnicas_e_comportamentais', 'demais_observacoes', 'viagens_requeridas', 'equipamentos_necessarios']


In [3]:
# Função para normalizar dados das vagas
def normalize_vagas(vagas_data):
    """Normaliza os dados das vagas para um formato estruturado"""
    vagas_list = []
    
    for vaga_id, vaga_info in vagas_data.items():
        try:
            basic_info = vaga_info.get('informacoes_basicas', {})
            profile_info = vaga_info.get('perfil_vaga', {})
            
            normalized_vaga = {
                'id_vaga': vaga_id,
                'titulo_vaga': basic_info.get('titulo_vaga', ''),
                'cliente': basic_info.get('cliente', ''),
                'tipo_contratacao': basic_info.get('tipo_contratacao', ''),
                'nivel_profissional': profile_info.get('nivel profissional', ''),
                'nivel_academico': profile_info.get('nivel_academico', ''),
                'nivel_ingles': profile_info.get('nivel_ingles', ''),
                'areas_atuacao': profile_info.get('areas_atuacao', ''),
                'competencias_tecnicas_requeridas': profile_info.get('competencia_tecnicas_e_comportamentais', ''),
                'principais_atividades': profile_info.get('principais_atividades', ''),
                'pais': profile_info.get('pais', ''),
                'estado': profile_info.get('estado', ''),
                'cidade': profile_info.get('cidade', '')
            }
            vagas_list.append(normalized_vaga)
        except Exception as e:
            print(f"Erro ao processar vaga {vaga_id}: {e}")
            continue
    
    return pd.DataFrame(vagas_list)

# Função para normalizar dados dos candidatos
def normalize_applicants(applicants_data):
    """Normaliza os dados dos candidatos"""
    applicants_list = []
    
    for candidate_id, candidate_info in applicants_data.items():
        try:
            basic_info = candidate_info.get('infos_basicas', {})
            personal_info = candidate_info.get('informacoes_pessoais', {})
            academic_info = candidate_info.get('formacao_academica', {})
            professional_info = candidate_info.get('informacoes_profissionais', {})
            
            normalized_candidate = {
                'codigo_candidato': candidate_id,
                'nome': basic_info.get('nome', ''),
                'email': basic_info.get('email', ''),
                'telefone': basic_info.get('telefone', ''),
                'data_nascimento': personal_info.get('data_nascimento', ''),
                'estado_civil': personal_info.get('estado_civil', ''),
                'pcd': personal_info.get('pcd', ''),
                'nivel_academico': academic_info.get('nivel_academico', '') if academic_info else '',
                'area_formacao': academic_info.get('area_formacao', '') if academic_info else '',
                'nivel_ingles': professional_info.get('nivel_ingles', '') if professional_info else '',
                'conhecimentos_tecnicos': professional_info.get('conhecimentos_tecnicos', '') if professional_info else '',
                'area_de_atuacao': professional_info.get('area_atuacao', '') if professional_info else ''
            }
            applicants_list.append(normalized_candidate)
        except Exception as e:
            print(f"Erro ao processar candidato {candidate_id}: {e}")
            continue
    
    return pd.DataFrame(applicants_list)

# Processar os dados
print("Processando dados das vagas...")
df_vagas = normalize_vagas(vagas_data)

print("Processando dados dos candidatos...")
df_candidates = normalize_applicants(applicants_data)

print(f"\nDataFrame Vagas: {df_vagas.shape}")
print(f"DataFrame Candidatos: {df_candidates.shape}")

# Visualizar primeiras linhas
print("\n=== Primeiras vagas ===")
print(df_vagas.head(3)[['id_vaga', 'titulo_vaga', 'nivel_profissional', 'areas_atuacao']])

Processando dados das vagas...
Processando dados dos candidatos...

DataFrame Vagas: (14081, 13)
DataFrame Candidatos: (42482, 12)

=== Primeiras vagas ===
  id_vaga             titulo_vaga nivel_profissional  \
0    5185        Operation Lead -             Sênior   
1    5184  Consultor PP/QM Sênior             Sênior   
2    5183   ANALISTA PL/JR C/ SQL           Analista   

                       areas_atuacao  
0       TI - Sistemas e Ferramentas-  
1  TI - Desenvolvimento/Programação-  
2       TI - Sistemas e Ferramentas-  


In [4]:
# Função para processar prospects e criar variável alvo
def process_prospects(prospects_data):
    """Processa os dados de prospects para extrair pares candidato-vaga com situação"""
    prospects_list = []
    
    for vaga_id, vaga_prospect in prospects_data.items():
        vaga_titulo = vaga_prospect.get('titulo', '')
        prospects = vaga_prospect.get('prospects', [])
        
        for prospect in prospects:
            try:
                # Extrair código do candidato (remover prefixos se existirem)
                codigo_candidato = prospect.get('codigo', '').strip()
                situacao = prospect.get('situacao_candidado', '').strip()
                
                prospect_record = {
                    'id_vaga': vaga_id,
                    'codigo_candidato': codigo_candidato,
                    'nome_candidato': prospect.get('nome', ''),
                    'situacao_candidado': situacao,
                    'data_candidatura': prospect.get('data_candidatura', ''),
                    'comentario': prospect.get('comentario', ''),
                    'recrutador': prospect.get('recrutador', ''),
                    # Criar variável alvo binária
                    'contratado': 1 if 'Contratado pela Decision' in situacao else 0
                }
                prospects_list.append(prospect_record)
            except Exception as e:
                print(f"Erro ao processar prospect da vaga {vaga_id}: {e}")
                continue
    
    return pd.DataFrame(prospects_list)

# Processar prospects
print("Processando dados dos prospects...")
df_prospects = process_prospects(prospects_data)

print(f"DataFrame Prospects: {df_prospects.shape}")
print(f"\nDistribuição da variável alvo 'contratado':")
print(df_prospects['contratado'].value_counts())
print(f"\nTaxa de contratação: {df_prospects['contratado'].mean():.2%}")

# Visualizar algumas situações de candidatos
print("\n=== Situações dos candidatos ===")
print(df_prospects['situacao_candidado'].value_counts().head(10))

Processando dados dos prospects...
DataFrame Prospects: (53759, 8)

Distribuição da variável alvo 'contratado':
contratado
0    51001
1     2758
Name: count, dtype: int64

Taxa de contratação: 5.13%

=== Situações dos candidatos ===
situacao_candidado
Prospect                          20021
Encaminhado ao Requisitante       16122
Inscrito                           3980
Não Aprovado pelo Cliente          3492
Contratado pela Decision           2758
Desistiu                           2349
Não Aprovado pelo RH               1765
Não Aprovado pelo Requisitante      765
Entrevista Técnica                  579
Sem interesse nesta vaga            576
Name: count, dtype: int64


## 2. Combinação e Preparação dos Dados

Vamos combinar os três DataFrames para criar um dataset unificado com pares candidato-vaga.

In [5]:
# Combinar os DataFrames
print("Combinando os dados...")

# Primeiro, fazer join prospects com vagas
df_combined = df_prospects.merge(
    df_vagas, 
    on='id_vaga', 
    how='inner'
)

print(f"Após join com vagas: {df_combined.shape}")

# Depois, fazer join com candidatos
df_final = df_combined.merge(
    df_candidates,
    on='codigo_candidato',
    how='inner',
    suffixes=('_vaga', '_candidato')
)

print(f"Dataset final: {df_final.shape}")
print(f"Taxa de contratação no dataset final: {df_final['contratado'].mean():.2%}")

# Verificar colunas disponíveis
print("\n=== Colunas disponíveis ===")
for i, col in enumerate(df_final.columns):
    print(f"{i+1:2d}. {col}")

# Visualizar algumas linhas
print("\n=== Amostra dos dados combinados ===")
sample_cols = ['id_vaga', 'titulo_vaga', 'nome_candidato', 'situacao_candidado', 'contratado']
print(df_final[sample_cols].head())

Combinando os dados...
Após join com vagas: (53735, 20)
Dataset final: (45071, 31)
Taxa de contratação no dataset final: 5.00%

=== Colunas disponíveis ===
 1. id_vaga
 2. codigo_candidato
 3. nome_candidato
 4. situacao_candidado
 5. data_candidatura
 6. comentario
 7. recrutador
 8. contratado
 9. titulo_vaga
10. cliente
11. tipo_contratacao
12. nivel_profissional
13. nivel_academico_vaga
14. nivel_ingles_vaga
15. areas_atuacao
16. competencias_tecnicas_requeridas
17. principais_atividades
18. pais
19. estado
20. cidade
21. nome
22. email
23. telefone
24. data_nascimento
25. estado_civil
26. pcd
27. nivel_academico_candidato
28. area_formacao
29. nivel_ingles_candidato
30. conhecimentos_tecnicos
31. area_de_atuacao

=== Amostra dos dados combinados ===
  id_vaga                                        titulo_vaga  \
0    4530                                CONSULTOR CONTROL M   
1    4530                                CONSULTOR CONTROL M   
2    4531  2021-2607395-PeopleSoft Applicat

## 3. Configuração do Banco de Dados PostgreSQL

Vamos criar estruturas de banco de dados PostgreSQL para armazenar os dados de forma eficiente e escalável. O PostgreSQL oferece recursos avançados como índices GIN para busca full-text e melhor performance para aplicações em produção.

In [ ]:
# CONFIGURAÇÃO DO BANCO DE DADOS POSTGRESQL
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd
from sqlalchemy import create_engine
import os
from pathlib import Path

print("=== CONFIGURANDO BANCO DE DADOS POSTGRESQL ===")

# Configurações do PostgreSQL - usando variáveis de ambiente
PG_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'database': os.getenv('DB_NAME', 'otimizador_entrevistas'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', 'postgres'),
    'port': int(os.getenv('DB_PORT', 5432))
}

print(f"🔗 Conectando ao PostgreSQL: {PG_CONFIG['host']}:{PG_CONFIG['port']}/{PG_CONFIG['database']}")

def create_pg_engine():
    """Cria engine SQLAlchemy para PostgreSQL"""
    connection_string = f"postgresql://{PG_CONFIG['user']}:{PG_CONFIG['password']}@{PG_CONFIG['host']}:{PG_CONFIG['port']}/{PG_CONFIG['database']}"
    return create_engine(connection_string)

def create_database_schema():
    """Cria as tabelas do banco PostgreSQL"""
    
    try:
        conn = psycopg2.connect(**PG_CONFIG)
        cursor = conn.cursor()
        
        print("📋 Criando tabelas PostgreSQL...")
        
        # Tabela de Vagas
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS vagas (
            id_vaga VARCHAR(50) PRIMARY KEY,
            titulo_vaga TEXT,
            cliente VARCHAR(200),
            tipo_contratacao VARCHAR(100),
            nivel_profissional VARCHAR(50),
            nivel_academico VARCHAR(50),
            nivel_ingles VARCHAR(50),
            areas_atuacao TEXT,
            competencias_tecnicas_requeridas TEXT,
            principais_atividades TEXT,
            pais VARCHAR(100),
            estado VARCHAR(100),
            cidade VARCHAR(100),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        ''')
        
        # Tabela de Candidatos
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS candidatos (
            codigo_candidato VARCHAR(50) PRIMARY KEY,
            nome VARCHAR(200),
            email VARCHAR(200),
            telefone VARCHAR(50),
            data_nascimento VARCHAR(20),
            estado_civil VARCHAR(50),
            pcd VARCHAR(10),
            nivel_academico VARCHAR(50),
            area_formacao VARCHAR(200),
            nivel_ingles VARCHAR(50),
            conhecimentos_tecnicos TEXT,
            area_de_atuacao VARCHAR(200),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        ''')
        
        # Tabela de Prospects (histórico de candidaturas)
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS prospects (
            id SERIAL PRIMARY KEY,
            id_vaga VARCHAR(50),
            codigo_candidato VARCHAR(50),
            nome_candidato VARCHAR(200),
            situacao_candidado VARCHAR(200),
            data_candidatura VARCHAR(20),
            comentario TEXT,
            recrutador VARCHAR(200),
            contratado INTEGER,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (id_vaga) REFERENCES vagas(id_vaga) ON DELETE CASCADE,
            FOREIGN KEY (codigo_candidato) REFERENCES candidatos(codigo_candidato) ON DELETE CASCADE
        )
        ''')
        
        # Tabela de Predições (log das predições do ML)
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS predicoes (
            id SERIAL PRIMARY KEY,
            id_vaga VARCHAR(50),
            codigo_candidato VARCHAR(50),
            tech_match_score REAL,
            academic_match VARCHAR(50),
            english_match VARCHAR(50),
            probabilidade_contratacao REAL,
            predicao_contratado INTEGER,
            modelo_versao VARCHAR(50),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (id_vaga) REFERENCES vagas(id_vaga) ON DELETE CASCADE,
            FOREIGN KEY (codigo_candidato) REFERENCES candidatos(codigo_candidato) ON DELETE CASCADE
        )
        ''')
        
        # Índices otimizados para PostgreSQL
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_vagas_nivel ON vagas(nivel_profissional)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_vagas_areas ON vagas USING gin(to_tsvector(\'portuguese\', areas_atuacao))')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_candidatos_area ON candidatos USING gin(to_tsvector(\'portuguese\', area_de_atuacao))')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_prospects_vaga ON prospects(id_vaga)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_prospects_candidato ON prospects(codigo_candidato)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_prospects_contratado ON prospects(contratado)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_predicoes_vaga ON predicoes(id_vaga)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_predicoes_data ON predicoes(created_at)')
        
        # Views úteis para PostgreSQL
        cursor.execute('''
        CREATE OR REPLACE VIEW vw_vagas_stats AS
        SELECT 
            v.id_vaga,
            v.titulo_vaga,
            v.cliente,
            v.nivel_profissional,
            v.areas_atuacao,
            COUNT(p.codigo_candidato) as total_candidatos,
            SUM(p.contratado) as total_contratados,
            CASE 
                WHEN COUNT(p.codigo_candidato) > 0 
                THEN ROUND(AVG(p.contratado) * 100, 2) 
                ELSE 0 
            END as taxa_contratacao
        FROM vagas v
        LEFT JOIN prospects p ON v.id_vaga = p.id_vaga
        GROUP BY v.id_vaga, v.titulo_vaga, v.cliente, v.nivel_profissional, v.areas_atuacao
        ''')
        
        cursor.execute('''
        CREATE OR REPLACE VIEW vw_candidatos_ranking AS
        SELECT 
            c.codigo_candidato,
            c.nome,
            c.area_de_atuacao,
            c.nivel_academico,
            COUNT(p.id_vaga) as total_candidaturas,
            SUM(p.contratado) as total_contratacoes,
            CASE 
                WHEN COUNT(p.id_vaga) > 0 
                THEN ROUND(AVG(p.contratado) * 100, 2) 
                ELSE 0 
            END as taxa_sucesso
        FROM candidatos c
        LEFT JOIN prospects p ON c.codigo_candidato = p.codigo_candidato
        GROUP BY c.codigo_candidato, c.nome, c.area_de_atuacao, c.nivel_academico
        ''')
        
        conn.commit()
        print("✅ Esquema PostgreSQL criado com sucesso!")
        
        # Verificar tabelas criadas
        cursor.execute("""
        SELECT tablename 
        FROM pg_tables 
        WHERE schemaname = 'public' 
        AND tablename IN ('vagas', 'candidatos', 'prospects', 'predicoes')
        """)
        tabelas = cursor.fetchall()
        print(f"📊 Tabelas criadas: {[t[0] for t in tabelas]}")
        
        # Verificar views criadas
        cursor.execute("""
        SELECT viewname 
        FROM pg_views 
        WHERE schemaname = 'public' 
        AND viewname LIKE 'vw_%'
        """)
        views = cursor.fetchall()
        print(f"👁️  Views criadas: {[v[0] for v in views]}")
        
    except Exception as e:
        print(f"❌ Erro ao criar esquema PostgreSQL: {e}")
        print("💡 Certifique-se de que:")
        print("   - PostgreSQL está rodando")
        print("   - Banco 'otimizador_entrevistas' existe")
        print("   - Credenciais estão corretas")
        print("   - psycopg2-binary está instalado: pip install psycopg2-binary")
        raise
    finally:
        if 'conn' in locals():
            conn.close()

# Testar conexão e criar esquema
try:
    create_database_schema()
except Exception as e:
    print(f"\n⚠️  Para configurar PostgreSQL:")
    print("1. Instale PostgreSQL: https://www.postgresql.org/download/")
    print("2. Crie o banco: createdb otimizador_entrevistas")
    print("3. Configure variáveis de ambiente:")
    print("   export DB_HOST=localhost")
    print("   export DB_NAME=otimizador_entrevistas") 
    print("   export DB_USER=postgres")
    print("   export DB_PASSWORD=sua_senha")
    print("4. Instale driver: pip install psycopg2-binary sqlalchemy")
    print(f"\nErro: {e}")

In [ ]:
# MIGRAÇÃO DOS DADOS JSON PARA POSTGRESQL
def migrate_data_to_postgresql():
    """Migra os dados dos DataFrames para o banco PostgreSQL"""
    
    try:
        engine = create_pg_engine()
        
        print("🚀 Migrando dados para PostgreSQL...")
        
        # Verificar se já existem dados
        with engine.connect() as conn:
            result = conn.execute("SELECT COUNT(*) FROM vagas").fetchone()
            vagas_count = result[0]
        
        if vagas_count > 0:
            print(f"⚠️  Banco já contém {vagas_count} vagas. Pulando migração.")
            print("   Para recriar os dados, execute: TRUNCATE TABLE prospects, candidatos, vagas RESTART IDENTITY CASCADE;")
            return
        
        # 1. Migrar Vagas
        print("📋 Migrando vagas para PostgreSQL...")
        df_vagas_clean = df_vagas.fillna('')  # Tratar valores nulos
        
        # Usar pandas to_sql com PostgreSQL
        rows_inserted = df_vagas_clean.to_sql(
            'vagas', 
            engine, 
            if_exists='append', 
            index=False,
            method='multi',  # Mais eficiente para grandes volumes
            chunksize=1000
        )
        print(f"✅ {len(df_vagas)} vagas migradas")
        
        # 2. Migrar Candidatos
        print("👥 Migrando candidatos para PostgreSQL...")
        df_candidates_clean = df_candidates.fillna('')  # Tratar valores nulos
        
        rows_inserted = df_candidates_clean.to_sql(
            'candidatos', 
            engine, 
            if_exists='append', 
            index=False,
            method='multi',
            chunksize=1000
        )
        print(f"✅ {len(df_candidates)} candidatos migrados")
        
        # 3. Migrar Prospects
        print("🎯 Migrando prospects para PostgreSQL...")
        df_prospects_clean = df_prospects.fillna('')  # Tratar valores nulos
        
        rows_inserted = df_prospects_clean.to_sql(
            'prospects', 
            engine, 
            if_exists='append', 
            index=False,
            method='multi',
            chunksize=1000
        )
        print(f"✅ {len(df_prospects)} prospects migrados")
        
        # Verificar migração usando pandas
        with engine.connect() as conn:
            vagas_df = pd.read_sql("SELECT COUNT(*) as count FROM vagas", conn)
            candidatos_df = pd.read_sql("SELECT COUNT(*) as count FROM candidatos", conn)
            prospects_df = pd.read_sql("SELECT COUNT(*) as count FROM prospects", conn)
            contratados_df = pd.read_sql("SELECT COUNT(*) as count FROM prospects WHERE contratado = 1", conn)
            
            vagas_db = vagas_df['count'].iloc[0]
            candidatos_db = candidatos_df['count'].iloc[0]
            prospects_db = prospects_df['count'].iloc[0]
            contratados_db = contratados_df['count'].iloc[0]
        
        print(f"\n📊 RESUMO DA MIGRAÇÃO POSTGRESQL:")
        print(f"   Vagas no banco: {vagas_db:,}")
        print(f"   Candidatos no banco: {candidatos_db:,}")
        print(f"   Prospects no banco: {prospects_db:,}")
        print(f"   Contratados: {contratados_db:,}")
        print(f"   Taxa de contratação: {contratados_db/prospects_db:.2%}")
        
        # Informações do banco PostgreSQL
        with engine.connect() as conn:
            # Tamanho das tabelas
            size_query = """
            SELECT 
                schemaname,
                tablename,
                pg_size_pretty(pg_total_relation_size(schemaname||'.'||tablename)) as size
            FROM pg_tables 
            WHERE schemaname = 'public' 
            AND tablename IN ('vagas', 'candidatos', 'prospects', 'predicoes')
            ORDER BY pg_total_relation_size(schemaname||'.'||tablename) DESC
            """
            sizes_df = pd.read_sql(size_query, conn)
            
            print(f"\n? TAMANHO DAS TABELAS:")
            for _, row in sizes_df.iterrows():
                print(f"   {row['tablename']}: {row['size']}")
        
        print(f"\n🎉 Migração PostgreSQL concluída com sucesso!")
        
    except Exception as e:
        print(f"❌ Erro na migração PostgreSQL: {e}")
        print(f"💡 Verifique se o PostgreSQL está rodando e as credenciais estão corretas")
        import traceback
        traceback.print_exc()

# Executar migração PostgreSQL
migrate_data_to_postgresql()

In [ ]:
# FUNÇÕES UTILITÁRIAS PARA CONSULTAS POSTGRESQL
def query_postgresql_stats():
    """Consulta estatísticas do banco PostgreSQL usando pandas"""
    
    try:
        engine = create_pg_engine()
        print("=== ESTATÍSTICAS DO BANCO POSTGRESQL ===")
        
        # Estatísticas básicas usando pandas
        with engine.connect() as conn:
            # Contadores básicos
            stats_query = """
            SELECT 
                'Total de Vagas' as metrica,
                (SELECT COUNT(*) FROM vagas) as valor
            UNION ALL
            SELECT 
                'Total de Candidatos',
                (SELECT COUNT(*) FROM candidatos)
            UNION ALL
            SELECT 
                'Total de Prospects',
                (SELECT COUNT(*) FROM prospects)
            UNION ALL
            SELECT 
                'Candidatos Contratados',
                (SELECT COUNT(*) FROM prospects WHERE contratado = 1)
            UNION ALL
            SELECT 
                'Vagas com Prospects',
                (SELECT COUNT(DISTINCT id_vaga) FROM prospects)
            """
            
            stats_df = pd.read_sql(stats_query, conn)
            
            for _, row in stats_df.iterrows():
                print(f"📊 {row['metrica']}: {row['valor']:,}")
        
        # Top 5 vagas com mais candidatos
        print(f"\n=== TOP 5 VAGAS COM MAIS CANDIDATOS ===")
        with engine.connect() as conn:
            top_vagas_query = '''
            SELECT 
                v.titulo_vaga,
                v.cliente,
                COUNT(p.codigo_candidato) as total_candidatos
            FROM vagas v
            LEFT JOIN prospects p ON v.id_vaga = p.id_vaga
            GROUP BY v.id_vaga, v.titulo_vaga, v.cliente
            ORDER BY total_candidatos DESC
            LIMIT 5
            '''
            
            top_vagas_df = pd.read_sql(top_vagas_query, conn)
            
            for _, row in top_vagas_df.iterrows():
                titulo = row['titulo_vaga'][:40]
                cliente = row['cliente'][:20]
                total = row['total_candidatos']
                print(f"   {titulo:<40} | {cliente:<20} | {total:,} candidatos")
        
        # Distribuição por nível profissional
        print(f"\n=== DISTRIBUIÇÃO POR NÍVEL PROFISSIONAL ===")
        with engine.connect() as conn:
            nivel_query = '''
            SELECT 
                nivel_profissional, 
                COUNT(*) as total
            FROM vagas
            WHERE nivel_profissional != '' AND nivel_profissional IS NOT NULL
            GROUP BY nivel_profissional
            ORDER BY total DESC
            '''
            
            nivel_df = pd.read_sql(nivel_query, conn)
            
            for _, row in nivel_df.iterrows():
                print(f"   {row['nivel_profissional']:<20} | {row['total']:,} vagas")
        
        # Taxa de contratação por área (usando HAVING otimizado para PostgreSQL)
        print(f"\n=== TAXA DE CONTRATAÇÃO POR ÁREA ===")
        with engine.connect() as conn:
            area_query = '''
            SELECT 
                v.areas_atuacao,
                COUNT(p.codigo_candidato) as total_candidatos,
                SUM(p.contratado) as contratados,
                ROUND(AVG(p.contratado::numeric) * 100, 2) as taxa_contratacao
            FROM vagas v
            LEFT JOIN prospects p ON v.id_vaga = p.id_vaga
            WHERE v.areas_atuacao != '' AND v.areas_atuacao IS NOT NULL
            GROUP BY v.areas_atuacao
            HAVING COUNT(p.codigo_candidato) >= 50
            ORDER BY taxa_contratacao DESC
            LIMIT 10
            '''
            
            area_df = pd.read_sql(area_query, conn)
            
            for _, row in area_df.iterrows():
                area = row['areas_atuacao'][:30]
                total_cand = row['total_candidatos']
                contratados = row['contratados']
                taxa = row['taxa_contratacao']
                print(f"   {area:<30} | {total_cand:,} candidatos | {contratados} contratados | {taxa}%")
                
    except Exception as e:
        print(f"❌ Erro ao consultar estatísticas PostgreSQL: {e}")

def validate_postgresql_consistency():
    """Valida a consistência dos dados no PostgreSQL"""
    
    try:
        engine = create_pg_engine()
        print("\n=== VALIDAÇÃO DE CONSISTÊNCIA POSTGRESQL ===")
        
        with engine.connect() as conn:
            # Verificar chaves estrangeiras órfãs
            orphan_query = '''
            SELECT COUNT(*) as orphans FROM prospects p
            WHERE p.id_vaga NOT IN (SELECT id_vaga FROM vagas)
            OR p.codigo_candidato NOT IN (SELECT codigo_candidato FROM candidatos)
            '''
            
            orphan_df = pd.read_sql(orphan_query, conn)
            orphans = orphan_df['orphans'].iloc[0]
            
            if orphans == 0:
                print("✅ Todas as chaves estrangeiras estão consistentes")
            else:
                print(f"⚠️  {orphans} prospects com chaves estrangeiras inválidas")
            
            # Verificar duplicatas usando PostgreSQL específico
            dup_query = '''
            SELECT 
                'vagas' as tabela,
                COUNT(*) - COUNT(DISTINCT id_vaga) as duplicatas
            FROM vagas
            UNION ALL
            SELECT 
                'candidatos',
                COUNT(*) - COUNT(DISTINCT codigo_candidato)
            FROM candidatos
            '''
            
            dup_df = pd.read_sql(dup_query, conn)
            total_dups = dup_df['duplicatas'].sum()
            
            if total_dups == 0:
                print("✅ Não há duplicatas nas chaves primárias")
            else:
                print(f"⚠️  {total_dups} duplicatas encontradas")
                for _, row in dup_df.iterrows():
                    if row['duplicatas'] > 0:
                        print(f"    {row['tabela']}: {row['duplicatas']} duplicatas")
            
            # Verificar valores nulos em campos críticos
            null_checks = {
                'vagas.titulo_vaga': "SELECT COUNT(*) FROM vagas WHERE titulo_vaga IS NULL OR titulo_vaga = ''",
                'candidatos.nome': "SELECT COUNT(*) FROM candidatos WHERE nome IS NULL OR nome = ''",
                'prospects.situacao_candidado': "SELECT COUNT(*) FROM prospects WHERE situacao_candidado IS NULL OR situacao_candidado = ''"
            }
            
            all_good = True
            for campo, query in null_checks.items():
                null_df = pd.read_sql(query, conn)
                nulls = null_df.iloc[0, 0]
                if nulls == 0:
                    print(f"✅ {campo}: Sem valores nulos/vazios")
                else:
                    print(f"⚠️  {campo}: {nulls} registros com valores nulos/vazios")
                    all_good = False
            
            # Verificar índices PostgreSQL
            indexes_query = '''
            SELECT 
                indexname,
                tablename,
                indexdef
            FROM pg_indexes 
            WHERE schemaname = 'public' 
            AND tablename IN ('vagas', 'candidatos', 'prospects', 'predicoes')
            ORDER BY tablename, indexname
            '''
            
            indexes_df = pd.read_sql(indexes_query, conn)
            print(f"\n📋 ÍNDICES POSTGRESQL: {len(indexes_df)} índices criados")
            
            # Mostrar índices GIN (específicos do PostgreSQL)
            gin_indexes = indexes_df[indexes_df['indexdef'].str.contains('gin', case=False)]
            if len(gin_indexes) > 0:
                print("🔍 Índices GIN para busca full-text:")
                for _, idx in gin_indexes.iterrows():
                    print(f"   {idx['tablename']}.{idx['indexname']}")
            
            if all_good:
                print("\n🎉 Banco PostgreSQL está consistente e pronto para uso!")
            else:
                print("\n⚠️  Algumas inconsistências encontradas - revisar dados se necessário")
                
    except Exception as e:
        print(f"❌ Erro na validação PostgreSQL: {e}")

def test_postgresql_advanced_features():
    """Testa recursos avançados do PostgreSQL"""
    
    try:
        engine = create_pg_engine()
        print("\n=== TESTANDO RECURSOS AVANÇADOS POSTGRESQL ===")
        
        with engine.connect() as conn:
            # Teste de busca full-text com GIN
            fulltext_query = '''
            SELECT 
                titulo_vaga,
                areas_atuacao,
                ts_rank(to_tsvector('portuguese', areas_atuacao), 
                       to_tsquery('portuguese', 'desenvolvimento')) as rank
            FROM vagas 
            WHERE to_tsvector('portuguese', areas_atuacao) @@ to_tsquery('portuguese', 'desenvolvimento')
            ORDER BY rank DESC
            LIMIT 5
            '''
            
            try:
                fulltext_df = pd.read_sql(fulltext_query, conn)
                print(f"✅ Busca full-text funcionando: {len(fulltext_df)} resultados para 'desenvolvimento'")
                if len(fulltext_df) > 0:
                    print("   Top resultado:", fulltext_df.iloc[0]['titulo_vaga'][:50])
            except Exception as e:
                print(f"⚠️  Busca full-text: {e}")
            
            # Teste das views criadas
            view_test_query = "SELECT COUNT(*) as total FROM vw_vagas_stats WHERE total_candidatos > 0"
            try:
                view_df = pd.read_sql(view_test_query, conn)
                print(f"✅ View vw_vagas_stats funcionando: {view_df['total'].iloc[0]} vagas com candidatos")
            except Exception as e:
                print(f"⚠️  View test: {e}")
            
            # Informações do banco PostgreSQL
            db_info_query = '''
            SELECT 
                current_database() as database_name,
                version() as postgresql_version,
                current_user as current_user,
                inet_server_addr() as server_ip,
                inet_server_port() as server_port
            '''
            
            db_info_df = pd.read_sql(db_info_query, conn)
            print(f"\n🗄️  INFORMAÇÕES DO BANCO:")
            print(f"   Database: {db_info_df['database_name'].iloc[0]}")
            print(f"   PostgreSQL: {db_info_df['postgresql_version'].iloc[0].split(' on ')[0]}")
            print(f"   Usuário: {db_info_df['current_user'].iloc[0]}")
            
    except Exception as e:
        print(f"❌ Erro nos testes avançados: {e}")

# Executar todas as consultas e validações PostgreSQL
query_postgresql_stats()
validate_postgresql_consistency()
test_postgresql_advanced_features()

In [ ]:
# CONFIGURAÇÃO DA APLICAÇÃO FLASK PARA POSTGRESQL
def generate_flask_postgresql_config():
    """Gera configuração para a aplicação Flask usar PostgreSQL"""
    
    # Código de configuração para app.py
    flask_config_code = '''
# Configuração PostgreSQL para app.py
import os
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from psycopg2.extras import RealDictCursor

# Configurações do PostgreSQL
PG_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'database': os.getenv('DB_NAME', 'otimizador_entrevistas'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', 'postgres'),
    'port': int(os.getenv('DB_PORT', 5432))
}

def create_pg_engine():
    """Cria engine SQLAlchemy para PostgreSQL"""
    connection_string = f"postgresql://{PG_CONFIG['user']}:{PG_CONFIG['password']}@{PG_CONFIG['host']}:{PG_CONFIG['port']}/{PG_CONFIG['database']}"
    return create_engine(connection_string, pool_pre_ping=True, pool_recycle=300)

def load_vagas_from_postgres():
    """Carrega vagas do PostgreSQL com filtro de prospects"""
    engine = create_pg_engine()
    query = """
    SELECT DISTINCT v.* 
    FROM vagas v 
    INNER JOIN prospects p ON v.id_vaga = p.id_vaga
    ORDER BY v.titulo_vaga
    """
    return pd.read_sql(query, engine)

def load_candidates_from_postgres():
    """Carrega candidatos do PostgreSQL"""
    engine = create_pg_engine()
    return pd.read_sql("SELECT * FROM candidatos ORDER BY nome", engine)

def load_prospects_from_postgres():
    """Carrega prospects do PostgreSQL"""
    engine = create_pg_engine()
    return pd.read_sql("SELECT * FROM prospects ORDER BY created_at DESC", engine)

def get_vagas_com_prospects():
    """Retorna set de vagas que têm prospects"""
    engine = create_pg_engine()
    query = "SELECT DISTINCT id_vaga FROM prospects"
    df = pd.read_sql(query, engine)
    return set(df['id_vaga'].tolist())

def save_predicao_to_postgres(predicao_data):
    """Salva predição no PostgreSQL"""
    engine = create_pg_engine()
    df = pd.DataFrame([predicao_data])
    df.to_sql('predicoes', engine, if_exists='append', index=False)
    print(f"✅ Predição salva no PostgreSQL para vaga {predicao_data.get('id_vaga')}")

def get_vaga_candidatos_postgres(id_vaga, limit=50):
    """Busca candidatos para uma vaga específica com paginação"""
    engine = create_pg_engine()
    query = """
    SELECT c.*, p.situacao_candidado, p.contratado
    FROM candidatos c
    INNER JOIN prospects p ON c.codigo_candidato = p.codigo_candidato
    WHERE p.id_vaga = %(id_vaga)s
    ORDER BY p.contratado DESC, c.nome
    LIMIT %(limit)s
    """
    return pd.read_sql(query, engine, params={'id_vaga': id_vaga, 'limit': limit})

# Função para busca otimizada com PostgreSQL
def search_vagas_postgres(search_term=None, nivel_filter=None, page=1, per_page=10):
    """Busca vagas com filtros e paginação otimizada"""
    engine = create_pg_engine()
    
    base_query = """
    SELECT v.*, COUNT(p.codigo_candidato) as total_candidatos
    FROM vagas v
    LEFT JOIN prospects p ON v.id_vaga = p.id_vaga
    WHERE 1=1
    """
    
    params = {}
    
    if search_term:
        base_query += " AND (v.titulo_vaga ILIKE %(search)s OR v.cliente ILIKE %(search)s)"
        params['search'] = f'%{search_term}%'
    
    if nivel_filter:
        base_query += " AND v.nivel_profissional = %(nivel)s"
        params['nivel'] = nivel_filter
    
    base_query += " GROUP BY v.id_vaga"
    base_query += " HAVING COUNT(p.codigo_candidato) > 0"
    base_query += " ORDER BY total_candidatos DESC"
    
    # Paginação
    offset = (page - 1) * per_page
    paginated_query = base_query + f" LIMIT {per_page} OFFSET {offset}"
    
    return pd.read_sql(paginated_query, engine, params=params)
'''
    
    # Salvar configuração
    config_file_path = Path("../app/postgresql_integration.py")
    with open(config_file_path, 'w', encoding='utf-8') as f:
        f.write(flask_config_code)
    
    print(f"? Configuração Flask-PostgreSQL gerada: {config_file_path}")
    
    # Arquivo .env de exemplo
    env_example = '''# Configurações PostgreSQL para .env
DB_HOST=localhost
DB_NAME=otimizador_entrevistas
DB_USER=postgres
DB_PASSWORD=sua_senha_aqui
DB_PORT=5432

# Para produção, use valores seguros:
# DB_HOST=seu_servidor_postgres
# DB_PASSWORD=senha_forte_aleatoria
'''
    
    env_file_path = Path("../app/.env.example")
    with open(env_file_path, 'w', encoding='utf-8') as f:
        f.write(env_example)
    
    print(f"📄 Arquivo .env.example criado: {env_file_path}")
    
    # Docker Compose para PostgreSQL
    docker_compose = '''version: '3.8'

services:
  postgresql:
    image: postgres:15-alpine
    container_name: otimizador_postgres
    environment:
      POSTGRES_DB: otimizador_entrevistas
      POSTGRES_USER: postgres
      POSTGRES_PASSWORD: postgres
    ports:
      - "5432:5432"
    volumes:
      - postgres_data:/var/lib/postgresql/data
      - ./init.sql:/docker-entrypoint-initdb.d/init.sql
    restart: unless-stopped

  app:
    build: .
    container_name: otimizador_app
    depends_on:
      - postgresql
    environment:
      DB_HOST: postgresql
      DB_NAME: otimizador_entrevistas
      DB_USER: postgres
      DB_PASSWORD: postgres
      DB_PORT: 5432
    ports:
      - "5000:5000"
    restart: unless-stopped

volumes:
  postgres_data:
'''
    
    docker_file_path = Path("../docker-compose.postgres.yml")
    with open(docker_file_path, 'w', encoding='utf-8') as f:
        f.write(docker_compose)
    
    print(f"? Docker Compose PostgreSQL criado: {docker_file_path}")
    
    print(f"\n📋 PRÓXIMOS PASSOS PARA INTEGRAÇÃO:")
    print("1. Instale dependências: pip install psycopg2-binary sqlalchemy")
    print("2. Configure .env: cp app/.env.example app/.env")
    print("3. Inicie PostgreSQL: docker-compose -f docker-compose.postgres.yml up -d postgresql")
    print("4. Execute este notebook para migrar os dados")
    print("5. Integre postgresql_integration.py no app.py")
    print("6. Teste a aplicação: python app/app.py")

# Gerar configurações Flask-PostgreSQL
print("\n=== GERANDO CONFIGURAÇÕES FLASK-POSTGRESQL ===")
generate_flask_postgresql_config()

## 4. Engenharia de Features

Vamos criar features relevantes para o modelo de Machine Learning.

In [6]:
# Preparar features básicas para o modelo
def prepare_features(df):
    """Prepara as features básicas para o modelo"""
    df_features = df.copy()
    
    # Tratar valores nulos e padronizar
    df_features = df_features.fillna('')
    
    # Criar feature de texto combinada (competências + conhecimentos)
    df_features['competencias_combinadas'] = (
        df_features['competencias_tecnicas_requeridas'].astype(str) + ' ' +
        df_features['conhecimentos_tecnicos'].astype(str) + ' ' +
        df_features['principais_atividades'].astype(str)
    ).str.lower().str.strip()
    
    # Verificar colunas disponíveis após merge
    print(f"Colunas disponíveis no DataFrame: {list(df_features.columns)}")
    
    # Definir features categóricas baseadas nas colunas que realmente existem
    potential_categorical = [
        'nivel_profissional', 
        'nivel_academico',  # Pode ser da vaga ou candidato
        'nivel_ingles',     # Pode ser da vaga ou candidato  
        'areas_atuacao', 
        'tipo_contratacao', 
        'area_de_atuacao'
    ]
    
    # Verificar quais colunas realmente existem
    available_categorical = []
    for feature in potential_categorical:
        if feature in df_features.columns:
            available_categorical.append(feature)
            # Limpar e padronizar
            df_features[feature] = df_features[feature].astype(str).str.strip().str.lower()
            df_features[feature] = df_features[feature].replace('', 'não_informado')
    
    print(f"Features categóricas identificadas: {available_categorical}")
    
    return df_features, available_categorical

# Preparar features básicas
print("Preparando features básicas...")
df_features, categorical_cols = prepare_features(df_final)

print(f"Features categóricas disponíveis: {categorical_cols}")
print(f"Tamanho do dataset: {df_features.shape}")

# Verificar distribuição de algumas features categóricas
print("\n=== Distribuição de features categóricas ===")
for col in categorical_cols[:3]:  # Mostrar só as primeiras 3
    if col in df_features.columns:
        print(f"\n{col}:")
        print(df_features[col].value_counts().head())

Preparando features...
Features categóricas disponíveis: ['nivel_profissional', 'nivel_academico_vaga', 'nivel_ingles_vaga', 'areas_atuacao', 'tipo_contratacao', 'nivel_academico_candidato', 'nivel_ingles_candidato', 'area_de_atuacao']
Tamanho do dataset: (45071, 32)

=== Distribuição de features categóricas ===

nivel_profissional:
nivel_profissional
sênior          17987
analista        14684
pleno            8597
júnior           1465
especialista      912
Name: count, dtype: int64

nivel_academico_vaga:
nivel_academico_vaga
ensino superior completo      33830
ensino médio completo          5391
ensino técnico completo        3501
ensino superior cursando       1441
ensino superior incompleto      520
Name: count, dtype: int64

nivel_ingles_vaga:
nivel_ingles_vaga
básico           14913
nenhum           11230
avançado          8736
fluente           4933
intermediário     4561
Name: count, dtype: int64


### 4.2. Features Avançadas

Agora vamos criar features inteligentes que capturam a compatibilidade entre vagas e candidatos:

1. **Tech Match Score**: Compatibilidade técnica baseada em sobreposição de palavras-chave
2. **Academic Match**: Compatibilidade de nível acadêmico usando hierarquia
3. **English Match**: Compatibilidade de nível de inglês
4. **Combined Text**: Texto unificado para análise semântica com TF-IDF

In [ ]:
# CRIAÇÃO DE FEATURES AVANÇADAS
print("=== CRIANDO FEATURES AVANÇADAS ===")
print("Baseado na análise dos dados, vamos criar:")
print("- Competências técnicas requeridas vs Conhecimentos técnicos")
print("- Nível acadêmico (vaga vs candidato)")
print("- Nível de idiomas")
print("- Principais atividades")
print("- Features categóricas originais")

def create_enhanced_features(df):
    """Cria features aprimoradas usando todas as informações disponíveis"""
    
    # Verificar colunas disponíveis
    print(f"Colunas disponíveis no DataFrame: {list(df.columns)}")
    
    # 1. Match de competências técnicas
    def calculate_tech_match(row):
        """Calcula match entre competências requeridas e conhecimentos técnicos"""
        competencias_req = str(row.get('competencias_tecnicas_requeridas', '')).lower()
        conhecimentos = str(row.get('conhecimentos_tecnicos', '')).lower()
        
        if not competencias_req or competencias_req == 'nan':
            return 0.0
        if not conhecimentos or conhecimentos == 'nan':
            return 0.0
            
        # Palavras-chave técnicas
        comp_words = set(competencias_req.split())
        conhec_words = set(conhecimentos.split())
        
        if len(comp_words) == 0:
            return 0.0
            
        # Interseção / união
        intersection = comp_words.intersection(conhec_words)
        match_score = len(intersection) / len(comp_words)
        
        return round(match_score, 2)
    
    # 2. Match de nível acadêmico
    def calculate_academic_match(row):
        """Verifica compatibilidade do nível acadêmico"""
        # Usar nomes corretos das colunas após merge
        nivel_vaga = str(row.get('nivel_academico', '')).lower()  # Da vaga
        nivel_candidato = str(row.get('nivel_academico_candidato', 
                                   row.get('nivel_academico', ''))).lower()  # Do candidato
        
        # Hierarquia acadêmica
        hierarchy = {
            'ensino médio': 1,
            'técnico': 2,
            'tecnólogo': 3,
            'superior': 4,
            'pós-graduação': 5,
            'mestrado': 6,
            'doutorado': 7
        }
        
        vaga_level = hierarchy.get(nivel_vaga, 0)
        candidato_level = hierarchy.get(nivel_candidato, 0)
        
        if vaga_level == 0 or candidato_level == 0:
            return 'indefinido'
        elif candidato_level >= vaga_level:
            return 'compatível'
        else:
            return 'insuficiente'
    
    # 3. Match de inglês
    def calculate_english_match(row):
        """Verifica compatibilidade do nível de inglês"""
        # Usar nomes corretos das colunas após merge
        nivel_vaga = str(row.get('nivel_ingles', '')).lower()  # Da vaga
        nivel_candidato = str(row.get('nivel_ingles_candidato',
                                   row.get('nivel_ingles', ''))).lower()  # Do candidato
        
        # Hierarquia de inglês
        hierarchy = {
            'nenhum': 0,
            'básico': 1,
            'intermediário': 2,
            'avançado': 3,
            'fluente': 4,
            'nativo': 5
        }
        
        vaga_level = hierarchy.get(nivel_vaga, -1)
        candidato_level = hierarchy.get(nivel_candidato, -1)
        
        if vaga_level == -1 or candidato_level == -1:
            return 'indefinido'
        elif candidato_level >= vaga_level:
            return 'compatível'
        else:
            return 'insuficiente'
    
    # Aplicar as funções
    df_enhanced = df.copy()
    
    print("Calculando match de competências técnicas...")
    df_enhanced['tech_match_score'] = df.apply(calculate_tech_match, axis=1)
    
    print("Calculando compatibilidade acadêmica...")
    df_enhanced['academic_match'] = df.apply(calculate_academic_match, axis=1)
    
    print("Calculando compatibilidade de inglês...")
    df_enhanced['english_match'] = df.apply(calculate_english_match, axis=1)
    
    # 4. Criar texto combinado para análise semântica
    def create_combined_text(row):
        """Combina textos relevantes para análise"""
        parts = [
            str(row.get('competencias_tecnicas_requeridas', '')),
            str(row.get('conhecimentos_tecnicos', '')),
            str(row.get('principais_atividades', '')),
            str(row.get('area_de_atuacao', ''))
        ]
        return ' '.join([p for p in parts if p and p != 'nan']).lower().strip()
    
    df_enhanced['combined_text'] = df.apply(create_combined_text, axis=1)
    
    return df_enhanced

# Aplicar melhorias no dataset preparado
df_enhanced = create_enhanced_features(df_features)

print(f"\nDataset aprimorado: {df_enhanced.shape}")
print("\nNovas features criadas:")
print("- tech_match_score: Score de match técnico (0.0 a 1.0)")
print("- academic_match: Compatibilidade acadêmica")
print("- english_match: Compatibilidade de inglês")
print("- combined_text: Texto combinado para análise semântica")

# Verificar as novas features
print(f"\n=== ESTATÍSTICAS DAS NOVAS FEATURES ===")
print(f"Tech match score - Média: {df_enhanced['tech_match_score'].mean():.2f}")
print(f"Tech match score - Range: {df_enhanced['tech_match_score'].min():.2f} a {df_enhanced['tech_match_score'].max():.2f}")

print(f"\nDistribuição Academic Match:")
print(df_enhanced['academic_match'].value_counts())

print(f"\nDistribuição English Match:")
print(df_enhanced['english_match'].value_counts())

print(f"\nAmostra dos dados aprimorados:")
sample_cols = ['nome_candidato', 'titulo_vaga', 'tech_match_score', 'academic_match', 'english_match', 'contratado']
available_cols = [col for col in sample_cols if col in df_enhanced.columns]
if available_cols:
    print(df_enhanced[available_cols].head())

## 5. Pipeline de Machine Learning

Agora vamos criar um pipeline completo que combina:
- **Pré-processamento automático** de features numéricas, categóricas e texto
- **Modelo Random Forest** otimizado para o problema de classificação
- **Validação cruzada** para avaliar a performance

In [ ]:
# CRIANDO PIPELINE DE MACHINE LEARNING COMPLETO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

print("=== CRIANDO PIPELINE COMPLETO ===")

# Selecionar features para o modelo
def select_features_for_model(df):
    """Seleciona features para o modelo"""
    
    # Features numéricas
    numeric_features = ['tech_match_score']
    
    # Features categóricas
    categorical_features = [
        'nivel_profissional',
        'areas_atuacao', 
        'area_de_atuacao',
        'academic_match',
        'english_match'
    ]
    
    # Feature de texto
    text_features = ['combined_text']
    
    # Verificar quais features existem no dataframe
    available_numeric = [f for f in numeric_features if f in df.columns]
    available_categorical = [f for f in categorical_features if f in df.columns]
    available_text = [f for f in text_features if f in df.columns]
    
    print(f"Features numéricas disponíveis: {available_numeric}")
    print(f"Features categóricas disponíveis: {available_categorical}")
    print(f"Features de texto disponíveis: {available_text}")
    
    return available_numeric, available_categorical, available_text

# Preparar dados para treinamento
numeric_cols, categorical_cols, text_cols = select_features_for_model(df_enhanced)

# Criar features X e target y
all_feature_cols = numeric_cols + categorical_cols + text_cols
X_enhanced = df_enhanced[all_feature_cols].copy()
y_enhanced = df_enhanced['contratado'].copy()

print(f"\nDataset para treinamento: {X_enhanced.shape}")
print(f"Distribuição do target: {y_enhanced.value_counts().to_dict()}")

# Criar transformadores
print("\nCriando transformadores...")

# Para features numéricas
numeric_transformer = StandardScaler()

# Para features categóricas
categorical_transformer = OneHotEncoder(
    handle_unknown='ignore',
    sparse_output=False
)

# Para features de texto
text_transformer = TfidfVectorizer(
    max_features=50,  # Limitado para dataset pequeno
    stop_words='english',
    ngram_range=(1, 2),
    min_df=1,  # Aceitar termos que aparecem pelo menos 1 vez
    lowercase=True
)

# Criar ColumnTransformer
transformers = []

if numeric_cols:
    transformers.append(('num', numeric_transformer, numeric_cols))
if categorical_cols:
    transformers.append(('cat', categorical_transformer, categorical_cols))
if text_cols:
    transformers.append(('text', text_transformer, text_cols[0]))  # TfidfVectorizer espera uma string

if not transformers:
    raise ValueError("Nenhuma feature válida encontrada!")

preprocessor_enhanced = ColumnTransformer(
    transformers=transformers,
    remainder='drop'
)

# Criar pipeline completo
pipeline_enhanced = Pipeline([
    ('preprocessor', preprocessor_enhanced),
    ('classifier', RandomForestClassifier(
        n_estimators=20,  # Número de árvores
        random_state=42,
        class_weight='balanced',  # Para lidar com desbalanceamento
        max_depth=5,  # Profundidade das árvores
        min_samples_split=2,
        min_samples_leaf=1
    ))
])

print(f"Pipeline criado com {len(transformers)} tipos de transformadores!")

# Treinar modelo
print("\nTreinando modelo...")
try:
    pipeline_enhanced.fit(X_enhanced, y_enhanced)
    print("✅ Modelo treinado com sucesso!")
    
    # Fazer predições no conjunto de treino
    predictions_enhanced = pipeline_enhanced.predict(X_enhanced)
    probabilities_enhanced = pipeline_enhanced.predict_proba(X_enhanced)
    
    # Avaliar desempenho
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
    
    accuracy_enhanced = accuracy_score(y_enhanced, predictions_enhanced)
    print(f"\nAcurácia do modelo: {accuracy_enhanced:.3f}")
    
    print("\nRelatório de classificação:")
    print(classification_report(y_enhanced, predictions_enhanced))
    
    print("\nMatriz de confusão:")
    print(confusion_matrix(y_enhanced, predictions_enhanced))
    
    # Validação cruzada (se possível)
    if len(X_enhanced) >= 3:
        try:
            cv_scores = cross_val_score(pipeline_enhanced, X_enhanced, y_enhanced, cv=3, scoring='accuracy')
            print(f"\nValidação cruzada (CV=3): {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")
        except Exception as cv_error:
            print(f"\nValidação cruzada não possível: {cv_error}")
    
    # Mostrar importância das features
    print(f"\n=== ANÁLISE DAS FEATURES ===")
    try:
        feature_importance = pipeline_enhanced.named_steps['classifier'].feature_importances_
        print(f"Número de features após transformação: {len(feature_importance)}")
        
        # Features mais importantes
        if len(feature_importance) > 0:
            top_indices = np.argsort(feature_importance)[-10:][::-1]
            print("\nTop 10 features mais importantes:")
            for i, idx in enumerate(top_indices):
                print(f"{i+1:2d}. Feature_{idx:<10} {feature_importance[idx]:.3f}")
    except Exception as e:
        print(f"Não foi possível calcular importância das features: {e}")
    
except Exception as e:
    print(f"❌ Erro no treinamento do modelo: {e}")
    import traceback
    traceback.print_exc()

## 6. Serialização e Teste do Modelo

Agora vamos salvar o modelo treinado para uso em produção e testá-lo com exemplos reais para validar seu funcionamento.

In [ ]:
# SALVAR MODELO E CRIAR FUNÇÃO DE TESTE
import joblib
import json
import os

print("=== SALVANDO MODELO PARA PRODUÇÃO ===")

# Criar diretório se não existir
os.makedirs('../app/models', exist_ok=True)

# Salvar pipeline treinado
model_path = '../app/models/pipeline_aprimorado.joblib'
joblib.dump(pipeline_enhanced, model_path)
print(f"✅ Modelo salvo em: {model_path}")

# Salvar metadados do modelo
model_metadata = {
    'model_version': '2.0_enhanced',
    'features': {
        'numeric': numeric_cols,
        'categorical': categorical_cols,
        'text': text_cols
    },
    'total_features_after_transform': len(pipeline_enhanced.named_steps['classifier'].feature_importances_),
    'model_type': 'RandomForestClassifier_Enhanced',
    'training_date': str(pd.Timestamp.now()),
    'dataset_size': len(X_enhanced),
    'accuracy': accuracy_enhanced,
    'class_distribution': y_enhanced.value_counts().to_dict(),
    'feature_engineering': [
        'tech_match_score: Score de compatibilidade técnica (0.0-1.0)',
        'academic_match: Compatibilidade acadêmica (compatível/insuficiente/indefinido)',
        'english_match: Compatibilidade de inglês (compatível/insuficiente/indefinido)',
        'combined_text: Análise semântica de competências e atividades'
    ]
}

metadata_path = '../app/models/model_metadata_enhanced.json'
with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(model_metadata, f, indent=2, ensure_ascii=False)

print(f"✅ Metadados salvos em: {metadata_path}")

# FUNÇÃO DE TESTE DO MODELO
def test_model_with_example(vaga_data, candidato_data):
    """Testa o modelo com dados de entrada"""
    
    print(f"\n=== TESTE DO MODELO ===")
    print(f"Vaga: {vaga_data.get('titulo_vaga', 'N/A')}")
    print(f"Candidato: {candidato_data.get('nome', 'N/A')}")
    
    # Simular processamento como seria feito na aplicação
    test_row = {**vaga_data, **candidato_data}
    
    # Aplicar mesma engenharia de features
    def calculate_tech_match_test(competencias_req, conhecimentos):
        comp_req = str(competencias_req).lower() if competencias_req else ''
        conhec = str(conhecimentos).lower() if conhecimentos else ''
        
        if not comp_req or not conhec:
            return 0.0
            
        comp_words = set(comp_req.split())
        conhec_words = set(conhec.split())
        
        if len(comp_words) == 0:
            return 0.0
            
        intersection = comp_words.intersection(conhec_words)
        return len(intersection) / len(comp_words)
    
    def calculate_academic_match_test(nivel_vaga, nivel_candidato):
        hierarchy = {
            'ensino médio': 1, 'técnico': 2, 'tecnólogo': 3,
            'superior': 4, 'pós-graduação': 5, 'mestrado': 6, 'doutorado': 7
        }
        
        vaga_level = hierarchy.get(str(nivel_vaga).lower(), 0)
        candidato_level = hierarchy.get(str(nivel_candidato).lower(), 0)
        
        if vaga_level == 0 or candidato_level == 0:
            return 'indefinido'
        return 'compatível' if candidato_level >= vaga_level else 'insuficiente'
    
    def calculate_english_match_test(nivel_vaga, nivel_candidato):
        hierarchy = {
            'nenhum': 0, 'básico': 1, 'intermediário': 2,
            'avançado': 3, 'fluente': 4, 'nativo': 5
        }
        
        vaga_level = hierarchy.get(str(nivel_vaga).lower(), -1)
        candidato_level = hierarchy.get(str(nivel_candidato).lower(), -1)
        
        if vaga_level == -1 or candidato_level == -1:
            return 'indefinido'
        return 'compatível' if candidato_level >= vaga_level else 'insuficiente'
    
    # Criar features de teste
    test_features = {
        'tech_match_score': calculate_tech_match_test(
            test_row.get('competencias_tecnicas_requeridas'),
            test_row.get('conhecimentos_tecnicos')
        ),
        'nivel_profissional': str(test_row.get('nivel_profissional', '')).lower(),
        'areas_atuacao': str(test_row.get('areas_atuacao', '')).lower(),
        'area_de_atuacao': str(test_row.get('area_de_atuacao', '')).lower(),
        'academic_match': calculate_academic_match_test(
            test_row.get('nivel_academico'),
            test_row.get('nivel_academico_candidato', test_row.get('nivel_academico'))
        ),
        'english_match': calculate_english_match_test(
            test_row.get('nivel_ingles'),
            test_row.get('nivel_ingles_candidato', test_row.get('nivel_ingles'))
        ),
        'combined_text': ' '.join([
            str(test_row.get('competencias_tecnicas_requeridas', '')),
            str(test_row.get('conhecimentos_tecnicos', '')),
            str(test_row.get('principais_atividades', '')),
            str(test_row.get('area_de_atuacao', ''))
        ]).lower().strip()
    }
    
    # Criar DataFrame para predição
    test_df = pd.DataFrame([test_features])
    
    # Fazer predição
    prediction = pipeline_enhanced.predict(test_df)[0]
    probability = pipeline_enhanced.predict_proba(test_df)[0]
    
    # Mostrar resultados
    print(f"\nFeatures calculadas:")
    for key, value in test_features.items():
        print(f"  {key}: {value}")
    
    print(f"\n📊 RESULTADO DA PREDIÇÃO:")
    print(f"   Predição: {'CONTRATADO' if prediction == 1 else 'NÃO CONTRATADO'}")
    print(f"   Probabilidade de contratação: {probability[1]:.1%}")
    print(f"   Probabilidade de não contratação: {probability[0]:.1%}")
    print(f"   Score de compatibilidade: {probability[1]*100:.1f}/100")
    
    return {
        'prediction': int(prediction),
        'probability_hired': float(probability[1]),
        'probability_not_hired': float(probability[0]),
        'match_score': float(probability[1] * 100),
        'features': test_features
    }

print("\n✅ Modelo salvo e função de teste criada com sucesso!")

In [ ]:
# TESTE PRÁTICO COM EXEMPLO REAL
print("\n" + "="*70)
print("TESTANDO MODELO COM EXEMPLO PRÁTICO")
print("="*70)

# Dados de teste baseados nos dados reais do dataset
vaga_exemplo = {
    'titulo_vaga': 'Desenvolvedor Python Sênior',
    'competencias_tecnicas_requeridas': 'Python Django Flask PostgreSQL AWS Docker',
    'nivel_profissional': 'sênior',
    'areas_atuacao': 'TI - Desenvolvimento',
    'principais_atividades': 'desenvolvimento backend apis rest microservices',
    'nivel_academico': 'superior',
    'nivel_ingles': 'intermediário'
}

candidato_exemplo = {
    'nome': 'João Silva',
    'conhecimentos_tecnicos': 'Python Django PostgreSQL Docker Kubernetes Git',
    'area_de_atuacao': 'Desenvolvimento Web',
    'nivel_academico_candidato': 'superior',
    'nivel_ingles_candidato': 'avançado'
}

# Executar teste
try:
    resultado = test_model_with_example(vaga_exemplo, candidato_exemplo)
    
    print(f"\n🎯 RESUMO DO TESTE:")
    print(f"   Score Final: {resultado['match_score']:.1f}%")
    print(f"   Recomendação: {'ALTA compatibilidade' if resultado['match_score'] >= 70 else 'MÉDIA compatibilidade' if resultado['match_score'] >= 50 else 'BAIXA compatibilidade'}")
    
    # Teste com candidato menos compatível
    print(f"\n" + "-"*50)
    print("TESTE COM CANDIDATO MENOS COMPATÍVEL")
    print("-"*50)
    
    candidato_menos_compativel = {
        'nome': 'Maria Santos',
        'conhecimentos_tecnicos': 'Java Spring MySQL',
        'area_de_atuacao': 'Desenvolvimento Mobile',
        'nivel_academico_candidato': 'técnico',
        'nivel_ingles_candidato': 'básico'
    }
    
    resultado2 = test_model_with_example(vaga_exemplo, candidato_menos_compativel)
    print(f"\n🎯 RESUMO DO TESTE 2:")
    print(f"   Score Final: {resultado2['match_score']:.1f}%")
    print(f"   Recomendação: {'ALTA compatibilidade' if resultado2['match_score'] >= 70 else 'MÉDIA compatibilidade' if resultado2['match_score'] >= 50 else 'BAIXA compatibilidade'}")
    
    print(f"\n✅ TREINAMENTO CONCLUÍDO COM SUCESSO!")
    print(f"📁 Modelo salvo em: ../app/models/pipeline_aprimorado.joblib")
    print(f"📋 Metadados salvos em: ../app/models/model_metadata_enhanced.json")
    
except Exception as e:
    print(f"❌ Erro no teste: {e}")
    import traceback
    traceback.print_exc()